In [1]:
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE RecordWildCards #-}
{-# LANGUAGE TemplateHaskell #-}

import AdventOfCode
import Control.Lens

import qualified Data.Attoparsec.ByteString.Char8 as A
import qualified Data.Map.Strict as M
import qualified Data.Vector as V

input <- dayLines 18

In [2]:
type Reg = Char
type ValueReg = Either Int Reg

parseReg = A.satisfy A.isAlpha_ascii
parseValue = A.signed A.decimal

parseValueReg :: A.Parser ValueReg
parseValueReg = A.eitherP parseValue parseReg

In [3]:
data Instr 
    = Snd ValueReg 
    | Set Reg ValueReg
    | Add Reg ValueReg
    | Mul Reg ValueReg
    | Mod Reg ValueReg
    | Rcv Reg
    | Jgz ValueReg ValueReg
    deriving (Eq, Show)

parseInstr = A.choice
    [ Snd <$> ("snd" *> pVR)
    , Set <$> ("set" *> pR)  <*> pVR
    , Add <$> ("add" *> pR)  <*> pVR
    , Mul <$> ("mul" *> pR)  <*> pVR
    , Mod <$> ("mod" *> pR)  <*> pVR
    , Rcv <$> ("rcv" *> pR)
    , Jgz <$> ("jgz" *> pVR) <*> pVR
    ]
    where
        pVR = A.space *> parseValueReg
        pR  = A.space *> parseReg

registerMap = M.fromList $ zip ['a'..'p'] (repeat 0)
instrVec    = V.fromList $ map (parsed parseInstr) input

In [4]:
data MachineState = MachineState
    { _registers :: M.Map Char Int
    , _counter   :: Int
    } deriving (Eq, Show)

getValueReg registers valueReg = case valueReg of
    Left v -> v
    Right r -> registers M.! r

executeInstr instr ms@MachineState{..} = case instr of
    Snd     vr -> ms{_counter = _counter + 1}
    Set reg vr -> ms{_registers = M.insert reg    (getValueReg _registers vr)      _registers, _counter = _counter + 1}
    Add reg vr -> ms{_registers = M.adjust      (+(getValueReg _registers vr)) reg _registers, _counter = _counter + 1}
    Mul reg vr -> ms{_registers = M.adjust      (*(getValueReg _registers vr)) reg _registers, _counter = _counter + 1}
    Mod reg vr -> ms{_registers = M.adjust (`mod` (getValueReg _registers vr)) reg _registers, _counter = _counter + 1}
    Jgz vr vr' -> if (getValueReg _registers vr) > 0
        then ms{_counter = _counter + (getValueReg _registers vr')}
        else ms{_counter = _counter + 1}
    other -> error $ "don't know how to execute " ++ show other

In [5]:
loop instrs ms@MachineState{..} lastFreq | _counter > V.length instrs || _counter < 0 = lastFreq
loop instrs ms@MachineState{..} lastFreq = case instrs V.! _counter of
    Snd vr -> loop instrs (executeInstr (Snd vr) ms) (getValueReg _registers vr)
    Rcv r  -> if (_registers M.! r) /= 0 then lastFreq else loop instrs ms{_counter = _counter + 1} lastFreq
    curr   -> loop instrs (executeInstr curr ms) lastFreq

In [6]:
loop instrVec (MachineState registerMap 0) 0

3423

In [7]:
send msg cms@ConcMachineState{..} = cms{_mailbox = _mailbox ++ [msg]}

data LockedState = Running | Waiting deriving (Eq, Show)
data ConcMachineState = ConcMachineState
    { _ms          :: MachineState
    , _mailbox     :: [Int]
    , _lockedState :: LockedState
    }

executeInstr' instr cms@ConcMachineState{..} = case instr of
    Rcv reg -> if _mailbox == [] 
        then cms{_lockedState = Waiting}
        else let
            (msg, mailbox') = (head _mailbox, tail _mailbox)
            registers' = M.insert reg msg $ _registers _ms
            in cms{_ms = _ms{_registers = registers', _counter = _counter _ms + 1}, _mailbox = mailbox', _lockedState = Running}
    _ -> cms{_ms = executeInstr instr _ms}

loop' instrs cms0 cms1 count
    | (_lockedState cms0) == Waiting && (_lockedState cms1) == Waiting     = count
    | (_counter $ _ms cms0) > V.length instrs || (_counter $ _ms cms0) < 0 = count
    | (_counter $ _ms cms1) > V.length instrs || (_counter $ _ms cms1) < 0 = count
    | Snd vr <- instrs V.! (_counter $ _ms cms0) = let
        value = getValueReg (_registers $ _ms cms0) vr
        cms1' = send value cms1
        cms0' = executeInstr' (Snd vr) cms0
        in loop' instrs cms0' cms1' count
    | Snd vr <- instrs V.! (_counter $ _ms cms1) = let
        value = getValueReg (_registers $ _ms cms1) vr
        cms0' = send value cms0
        cms1' = executeInstr' (Snd vr) cms1
        in loop' instrs cms0' cms1' (count + 1)
    | otherwise = let
        instr0 = instrs V.! (_counter $ _ms cms0)
        instr1 = instrs V.! (_counter $ _ms cms1)
        cms0' = executeInstr' instr0 cms0
        cms1' = executeInstr' instr1 cms1
        in loop' instrs cms0' cms1' count

initialMs0 = ConcMachineState (MachineState                 registerMap  0) [] Running
initialMs1 = ConcMachineState (MachineState (M.insert 'p' 1 registerMap) 0) [] Running

loop' instrVec initialMs0 initialMs1 0

7493